https://stackoverflow.com/questions/53028624/row-wise-difference-in-two-list-in-pandas

In [2]:
from numpy import nan as NA
import pandas as pd
import collections

df = pd.DataFrame({'ID':['A','B','C','A','B','A','A','A','D','E','E','E'],
                   'Value': [1,2,3,4,3,5,2,3,7,2,3,9]})
#wrap all elements by group in a list
Changed_df=df.groupby('ID')['Value'].apply(list).reset_index() 
Changed_df=Changed_df.rename(columns={'Value' : 'Elements'})
Changed_df=Changed_df.reset_index(drop=True)



def flatten(l):
    for el in l:
        if isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

Changed_df["Elements_s"]=Changed_df['Elements'].shift()

#attempt 1: For loop
Changed_df["Diff"]=NA
Changed_df["count"]=0
Elements_so_far = []

#replace NA with empty list in columns that will go through list operations
for col in ["Elements","Elements_s","Diff"]:
    Changed_df[col] = Changed_df[col].apply(lambda d: d if isinstance(d, list) else [])

for idx,row in Changed_df.iterrows():
    diff = list(set(row['Elements']) - set(Elements_so_far))
    Changed_df.at[idx, "Diff"] = diff
    Elements_so_far.append(row['Elements'])
    Elements_so_far = flatten(Elements_so_far)
    Elements_so_far = list(set(Elements_so_far)) #keep unique elements
    Changed_df.loc[idx,"count"]=diff.__len__()

In [3]:
df

,ID,Value
0,A,1
1,B,2
2,C,3
3,A,4
4,B,3
5,A,5
6,A,2
7,A,3
8,D,7
9,E,2


In [4]:
Changed_df

,ID,Elements,Elements_s,Diff,count
0,A,"[1, 4, 5, 2, 3]",[],"[1, 2, 3, 4, 5]",5
1,B,"[2, 3]","[1, 4, 5, 2, 3]",[],0
2,C,[3],"[2, 3]",[],0
3,D,[7],[3],[7],1
4,E,"[2, 3, 9]",[7],[9],1
